In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from skimage.metrics import peak_signal_noise_ratio, structural_similarity as ssim

# ----------- PATHS -----------
TEST_IMG_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/test/img"

BASIC_MODEL_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"
UPD_MODEL_DIR   = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models/upd"

IMAGE_SIZE = (256, 256)

STYLES = ["blur", "night_vis", "poster", "outline"]

# ----------- LOAD TEST IMAGES -----------
def load_test_images():
    imgs = []
    names = []
    for fname in sorted(os.listdir(TEST_IMG_DIR)):
        if fname.lower().endswith((".jpg",".jpeg",".png",".bmp",".tiff")):
            path = os.path.join(TEST_IMG_DIR, fname)
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, IMAGE_SIZE)
            img = img.astype("float32") / 255.0
            imgs.append(img)
            names.append(fname)
    return np.array(imgs), names

test_imgs, test_names = load_test_images()
print("Loaded test:", test_imgs.shape)


# ----------- METRIC FUNCTIONS -----------
def compute_metrics(y_true, y_pred):
    mae = np.mean(np.abs(y_true - y_pred))
    mse = np.mean((y_true - y_pred)**2)
    psnr = peak_signal_noise_ratio(y_true, y_pred, data_range=1.0)
    ssim_value = ssim(y_true, y_pred, channel_axis=-1, data_range=1.0)
    return mae, mse, psnr, ssim_value


# ----------- EVALUATE MODELS -----------
results = []

for style in STYLES:
    print(f"\n=== Evaluating: {style.upper()} ===")

    # load basic and updated models
    basic_path = os.path.join(BASIC_MODEL_DIR, f"autoencoder_{style}.keras")
    upd_path   = os.path.join(UPD_MODEL_DIR, f"autoencoder_{style}_upd.keras")

    available_versions = []
    if os.path.exists(basic_path):
        available_versions.append(("basic", basic_path))
    if os.path.exists(upd_path):
        available_versions.append(("updated", upd_path))

    for version, path in available_versions:
        model = load_model(path)
        preds = model.predict(test_imgs)

        # compute metrics per image, then average
        style_mae, style_mse, style_psnr, style_ssim = [], [], [], []

        for y_true, y_pred in zip(test_imgs, preds):
            m1, m2, p, s = compute_metrics(y_true, y_pred)
            style_mae.append(m1)
            style_mse.append(m2)
            style_psnr.append(p)
            style_ssim.append(s)

        results.append({
            "style": style,
            "version": version,
            "MAE": np.mean(style_mae),
            "MSE": np.mean(style_mse),
            "PSNR": np.mean(style_psnr),
            "SSIM": np.mean(style_ssim),
        })


# ----------- SHOW RESULTS -----------
import pandas as pd

df_results = pd.DataFrame(results)
df_results
